# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [2]:
import pandas as pd

In [ ]:
# your code goes here
marketing_df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv")

In [13]:
#1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.
high_claim_df = marketing_df[marketing_df["Total Claim Amount"] > 1000]

In [18]:
#2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.
marketing_df[marketing_df["Response"] == "Yes"].groupby(["Gender","Policy Type"])["Total Claim Amount"].agg(["mean"])
#For males te Personal Auto calim amount is bigger, while the female amounts are larger in genereal in all policy types

mean
Gender Policy Type               
F      Corporate Auto  433.738499
       Personal Auto   452.965929
       Special Auto    453.280164
M      Corporate Auto  408.582459
       Personal Auto   457.010178
       Special Auto    429.527942

In [22]:
#3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.
marketing_df.groupby(["State"])["Customer"].agg(["count"]).reset_index()

,State,count
0,Arizona,1937
1,California,3552
2,Nevada,993
3,Oregon,2909
4,Washington,888


In [39]:
count_state = marketing_df.groupby(["State"])["Customer"].agg(["count"]).reset_index()
count_state[count_state['count'] > 500]

,State,count
0,Arizona,1937
1,California,3552
2,Nevada,993
3,Oregon,2909
4,Washington,888


In [36]:
#4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.
marketing_df.groupby(["Education","Gender"])["Customer Lifetime Value"].agg(["min", "max", "median"])
#The minimum increases with the education in both genders The median increases with education in males

min          max       median
Education            Gender                                       
Bachelor             F       1904.000852  73225.95652  5640.505303
                     M       1898.007675  67907.27050  5548.031892
College              F       1898.683686  61850.18803  5623.611187
                     M       1918.119700  61134.68307  6005.847375
Doctor               F       2395.570000  44856.11397  5332.462694
                     M       2267.604038  32677.34284  5577.669457
High School or Below F       2144.921535  55277.44589  6039.553187
                     M       1940.981221  83325.38119  6286.731006
Master               F       2417.777032  51016.06704  5729.855012
                     M       2272.307310  50568.25912  5579.099207

In [58]:
#5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.
marketing_df["month"] = pd.to_datetime(marketing_df["Effective To Date"], format="%m/%d/%y").dt.month
marketing_df.pivot_table(columns="month",index="State", values="Unnamed: 0", aggfunc="count" )


month,1,2
State,,
Arizona,1008,929
California,1918,1634
Nevada,551,442
Oregon,1565,1344
Washington,463,425


In [66]:
#6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

#*Hint:*
#- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
#- *Next, you will select the top 3 states with the highest number of policies sold.*
#- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

monthly_sales = marketing_df.groupby(["State", "month"])["Customer"].count().reset_index()

total_sales_by_state = monthly_sales.groupby("State")["Customer"].sum().reset_index()

top_states = total_sales_by_state.sort_values(by="Customer", ascending=False).head(3)

top_states_monthly_sales = monthly_sales[monthly_sales["State"].isin(top_states["State"])]

print(top_states_monthly_sales)

        State  month  Customer
0     Arizona      1      1008
1     Arizona      2       929
2  California      1      1918
3  California      2      1634
6      Oregon      1      1565
7      Oregon      2      1344
